# parcels regions

runs parcels on existing netcdf files

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import math
import sys
from pathlib import Path
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from parcels import FieldSet, ParticleSet
from datetime import timedelta

import utils
from utils import load_config, create_path, add_noise
from parcels_utils import get_file_info
from plot_utils import plot_particles_age

# ignore annoying deprecation warnings
import warnings
warnings.simplefilter("ignore", UserWarning)
import cartopy

# ignore divide by nan error that happens constantly with parcels
np.seterr(divide='ignore', invalid='ignore')

## configuration

change the contents of `configs` for the simulation configuration you want

In [ ]:
configs = [
    "parcels_configs/tijuana_interped.json",
#     "parcels_configs/tijuana_lin.json",
#     "parcels_configs/tijuana_lin_aggr.json",
#     "parcels_configs/tijuana_less.json",
#     "parcels_configs/tijuana_now.json",
#     "parcels_configs/tijuana_onerep.json",
#     "parcels_configs/tijuana_range.json",
]

loaded_configs = [load_config(path) for path in configs]
files = [get_file_info(cfg["netcdf_path"], cfg["resolution"], name=cfg["name"], parcels_cfg=cfg["parcels_config"]) for cfg in loaded_configs]

## Animated gif stuff and particle simulation

In [ ]:
# animation man very cool
# reference tutorial_Agulhasparticles
# needs ErrorCode for particle recovery
from operator import attrgetter
from parcels import ErrorCode, JITParticle, Variable, AdvectionRK4

max_v = 0.6 # for display purposes only, so the vector field colors don't change every iteration

class TimedParticle(JITParticle):
    lifetime = Variable("lifetime", initial=0, dtype=np.float32)
    spawntime = Variable("spawntime", initial=attrgetter("time"), dtype=np.float32)

    
def ParticleLifetime(particle, fieldset, time):
    particle.lifetime += particle.dt

    
def DeleteParticle(particle, fieldset, time):
    particle.delete()

    
def exec_save_pset(data, i, runtime, dt, zpad=3, ages=True, save_snapshot=True, exec_pset=True):
    """
    Saves a snapshot of a particle simulation and then executes.
    
    Args:
        data (dict)
        i (int)
        runtime (float): seconds
        dt (float): seconds
    """
    if len(data["pset"]) == 0:
        print("ParticleSet is empty. Passing...", file=sys.stderr)
        return
    
    if save_snapshot:
        days = (data["timerng"][1] - data["timerng"][0]) / np.timedelta64(1, 'D')
        field = None if ages else "vector"
        if "shown_domain" in data["cfg"]:
            dom = data["cfg"]["shown_domain"]
        else:
            dom = data["domain"]
        plot_particles_age(data["pset"], dom, field=field,
                           savefile=str(data["snap_path"])+"/particles"+str(i).zfill(zpad),
                           vmax=days, field_vmax=max_v)
    
    if exec_pset:
        # temporary - TODO make it only init once
        k_plifetime = data["pset"].Kernel(ParticleLifetime)

        data["pset"].execute(
            AdvectionRK4 + k_plifetime,
            runtime=timedelta(seconds=runtime),
            dt=timedelta(seconds=dt),
            recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
            output_file=data["pfile"]
        )
        
        
def parse_time_range(time_range, data):
    """
    Args:
        time_range (array-like): some array with 2 strings
        data (dict)
    """
    if time_range[0] == "START":
        t_start = data["timerng"][0]
    else:
        try:
            t_start = int(time_range[0])
        except ValueError:
            t_start = np.datetime64(time_range[0], "h")

    if time_range[1] == "END":
        t_end = data["timerng"][1]
    else:
        try:
            t_end = int(time_range[1])
        except ValueError:
            t_end = np.datetime64(time_range[1], "h")
            
    if isinstance(t_start, int) and isinstance(t_end, int):
        raise TypeError("Must have at least one date in the time range.")
    if isinstance(t_start, int):
        t_start = t_end - np.timedelta64(t_start, "h")
    if isinstance(t_end, int):
        t_end = t_start + np.timedelta64(t_end, "h")
        
    return t_start, t_end

### ParticleSet and spawn point setup

note about interpolation methods: only `linear` works if you want to use the FieldSet in a ParticleSet.

In [ ]:
part_path = create_path("particledata")

for f in files:
    cfg = f["cfg"]
    cfg["spawn_points"] = np.array(cfg["spawn_points"])
    
    if "time_range" not in cfg:
        t_start = f["timerng_secs"][0]
        t_end = f["timerng_secs"][1]
    else:
        t_start, t_end = parse_time_range(cfg["time_range"], f)
        t_start = (t_start - f["timerng"][0]) / np.timedelta64(1, "s")
        t_end = (t_end - f["timerng"][0]) / np.timedelta64(1, "s")
            
    if cfg["repeat_dt"] <= 0:
        repetitions = 1
    else:
        repetitions = int((t_end - t_start) / cfg["repeat_dt"])
    cfg["sim_start_sec"] = t_start
    cfg["sim_end_sec"] = t_end
    # the total number of particles that will exist in the simulation
    total = repetitions * cfg["particles_per_dt"]
    lat_arr = np.zeros(total)
    lon_arr = np.zeros(total)
    time_arr = np.zeros(total)
    for i in range(repetitions):
        time_arr[cfg["particles_per_dt"] * i:cfg["particles_per_dt"] * (i + 1)] = t_start + cfg["repeat_dt"] * i

    # randomly select spawn points from the given config
    sp_lat = cfg["spawn_points"].T[0][np.random.randint(0, len(cfg["spawn_points"]), total)]
    sp_lon = cfg["spawn_points"].T[1][np.random.randint(0, len(cfg["spawn_points"]), total)]
    # vary spawn locations using max_variation
    p_lats = add_noise(sp_lat, cfg["max_variation"])
    p_lons = add_noise(sp_lon, cfg["max_variation"])

    # set up ParticleSet and ParticleFile
    f["pset"] = ParticleSet(fieldset=f["fs"], pclass=TimedParticle, lon=p_lons, lat=p_lats, time=time_arr)
    save_path = f"{part_path}/particle_{f['name']}.nc"
    f["pfile"] = f["pset"].ParticleFile(save_path)
    print(f"Particle trajectories for {f['name']} will be saved to {save_path}")
    print(f"    total particles in simulation: {total}")

In [ ]:
[f["pset"].show(field="vector", vmax=max_v) for f in files]

### simulation setup and execution

simulation parameter setup

In [ ]:
# setting up times, intervals, and paths for simulation
for f in files:
    cfg = f["cfg"]
    cfg["snap_num"] = math.floor((cfg["sim_end_sec"] - cfg["sim_start_sec"]) / cfg["snapshot_interval"])
    cfg["last_int"] = cfg["sim_end_sec"] - (cfg["snap_num"] * cfg["snapshot_interval"] + cfg["sim_start_sec"])
    if cfg["last_int"] == 0:
        print("No last interval exists.")
        print(f"Num snapshots to save for {f['path']}: {cfg['snap_num'] + 1}")
    else:
        print(f"Num snapshots to save for {f['path']}: {cfg['snap_num'] + 2}")
    if cfg["snap_num"] >= 200:
        raise Exception(f"Too many snapshots ({cfg['snap_num']}).")
    f["snap_path"] = create_path(f"snapshots/{utils.filename_dict[f['res']]}/{f['name']}")
    print(f"Path to save snapshots to: {f['snap_path']}")
    # only clear directory if desired or actually saving images
    if cfg["save_snapshots"]:
        for p in f["snap_path"].glob("*.png"):
            p.unlink()

execution of all simulation configurations

In [ ]:
# show age or not (temporary)
age = False
# execution of simulation
for f in files:
    cfg = f["cfg"]
    for i in range(cfg["snap_num"]):
        exec_save_pset(f, i, cfg["snapshot_interval"], cfg["simulation_dt"], ages=age, save_snapshot=cfg["save_snapshots"])

    # save the second-to-last frame
    exec_save_pset(f, cfg["snap_num"], 0, 0, ages=age, save_snapshot=cfg["save_snapshots"], exec_pset=False)

    # run the last interval (the remainder) if needed
    if cfg["last_int"] != 0:
        exec_save_pset(f, cfg["snap_num"] + 1, cfg["last_int"], cfg["simulation_dt"], ages=age, save_snapshot=cfg["save_snapshots"])
        
    # run one more time with 0 runtime because for some reason
    # particles can be out-of-bounds without being deleted
    # ??? how does this happen ???
    # im not actually sure if this really does anything but hey its worth a shot
    exec_save_pset(f, 0, 0, 0, save_snapshot=False)
        
    f["pfile"].export()
    f["pfile"].close()

print("all simulations done and snapshots saved (if simulation was saving snapshots)")

### gif generation

don't have to run, requires [magick](https://imagemagick.org/index.php)

the gifs will be saved `snapshots/west_coast_xkm_hourly/` where xkm is the resolution

In [ ]:
import subprocess

gif_delay = 25 # ms

for f in files:
    if f["cfg"]["save_snapshots"]:
        magick_sp = subprocess.Popen(
            [
                "magick", "-delay", str(gif_delay),
                str(f["snap_path"]) + "/*.png", # path to the snapshots to stitch
                f"snapshots/{utils.filename_dict[f['res']]}/partsim_{f['name']}.gif" # path to save gif to
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True
        )
        stdout, stderr = magick_sp.communicate()
        print((stdout, stderr))